# Online Perceptron for Optical Character Recognition

In the Optical Character Recognition (OCR) we seek to predict, for a given handwritten digit's image, the corresponding number. Here, we simplify the OCR into a binary classification task. Specifically, we consider predictions for only two numbers 3 and 5. 
## Training Data
Includes 4888 rows (samples). Each sample/ row is in fact the target digit and a list of flattened gray-scale values from a 2d digital handwritten image with shape 28 × 28. -->  1 + 784  = 785 values. The first number is the digit’s label which is 3 or 5. The other 784 floating values are the grayscale values.
## Validation Data
 Includes 1629 rows. Each row obeys the same format given for the train set. This set will be used to select your best trained model.
 ## Test Data
 Includes 1629 rows. Each row contains only 784 numbers. The label column (the first column in the trianing and the validatin data) is omitted from each row here.


In [1]:
import time
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

## Data Preprocessing Module

In [6]:
class Preprocessing:
    
    def convert_into_perceptron_binary_classes(self, df):
        #change each row based on the condition
        
        final = []
        
        for idx in range(len(df)):
            if df[idx] == 3:
                final.append(1)
            else:
                final.append(-1)
        return pd.DataFrame(final)
        
    
    def preprocess_train_csv(self, path):
        raw_data = pd.read_csv(path, sep = ",", header = None)
        features = raw_data.iloc[:, 1:]
        
        targets = raw_data.loc[:,0]
        
        #normalize the features
        scaler = MinMaxScaler()
        scaler.fit(features)
        scaled_features = scaler.transform(features) 
        
        #bias of all ones
        bias_feature = pd.DataFrame(np.ones(len(features)).reshape((len(features),1)))
        
        #stack the new bias feature to the left of the regular features using the axis argument
        features_with_bias = pd.concat([bias_feature, features], axis = 1) #now has 785 features including the bias
        
        #convert the targets into -1 and +1 for perceptron operation        
        targets = self.convert_into_perceptron_binary_classes(targets)
        
        return scaler, scaled_features, targets
    
    def preprocess_validation_csv(self, scaler, path):
        raw_data = pd.read_csv(path, sep = ",", header = None)
        features = raw_data.iloc[:, 1:]
        targets = raw_data.loc[:,0]
        #transform the features using the parameters from the training data
        scaled_features = scaler.transform(features)
        
        #bias of all ones
        bias_feature = pd.DataFrame(np.ones(len(features)).reshape((len(features),1)))
        
        #stack the new bias feature to the left of the regular features using the axis argument
        features_with_bias = pd.concat([bias_feature, features], axis = 1) #now has 785 features including the bias
        
        
        
        #convert the targets into -1 and +1 for perceptron operation
        targets = self.convert_into_perceptron_binary_classes(targets)
        
        return  scaled_features, targets
        
        
        
    def preprocess_test_csv(self, scaler, path):
        features = pd.read_csv(path, sep = ",", header = None)
        
        
        #transform the features using the parameters from the training data
        scaled_features =  pd.DataFrame(scaler.transform(features))
        #bias of all ones
        bias_feature = pd.DataFrame(np.ones(len(scaled_features)).reshape((len(scaled_features),1)))
        
        #stack the new bias feature to the left of the regular features using the axis argument
        features_with_bias = pd.concat([bias_feature, scaled_features], axis = 1) #now has 785 features including the bias
        
        return features_with_bias
        
        

# Perceptron Module

In [ ]:
class Perceptron:
    
    def __init__(self):
        pass
    
    
    
    
class Vanilla_Perceptron(Perceptron):
    
    def __init__(self, max_iters, features_train, target_train, features_validation, targets_validation, features_test, targets_test):
        self.features_train = features_train
        self.targets_train = targets_train
        self.features_validation = features_validation
        self.targets_validation = targets_validation
        self.features_test = features_test
        self.targets_test = targets_test
    
    def train():
        
        #Initialize the weights and biases randomly
        weights = pd.DataFrame(np.zeros(self.features.shape[1]))
        biases = pd.DataFrame(np.zeros(self.features.shape[1]))
        
        iteration = 0
        
        while iteration < max_iters:
            
            #Get a shuffled list of indices
            shuffled_indices = np.random.permutation(len(self.features_train))
            
            for idx in range(len(shuffled_indices)):
                
                W_times_X = weights.dot(self.features_train)
                
                if self.targets_train[idx] * W_times_X <= 0:
                    
                    
        
        

## Training Module

In [7]:
#Prepare data
train_data_path = r"C:\Users\Being_Aerys\PycharmProjects\Machine_Learning_Algorithms_Collection\Supervised_Methods\Perceptron\Data\pa2_train.csv"
validation_data_path = r"C:\Users\Being_Aerys\PycharmProjects\Machine_Learning_Algorithms_Collection\Supervised_Methods\Perceptron\Data\pa2_valid.csv"
test_data_path = r"C:\Users\Being_Aerys\PycharmProjects\Machine_Learning_Algorithms_Collection\Supervised_Methods\Perceptron\Data\pa2_test.csv"

preprocessing = Preprocessing()

scaler, training_features, training_targets = preprocessing.preprocess_train_csv(train_data_path)
validation_features, validation_targets = preprocessing.preprocess_validation_csv(scaler, validation_data_path)
test_features = preprocessing.preprocess_test_csv(scaler, test_data_path)

DONE
